In [1]:
# user-friendly print
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Crystal structure generation

To generate reasonable crystal structure under a given space group with a specific chemical composition, basicly the following four steps are needed.

1. calculate possible Wyckoff configurations under a given space group for each chemical composition.
2. generate fraction positions for each element with given a Wyckoff configuration which is calculated from step 1), randomly.
3. generate lattice for the given space group which is used in step 1), randomly.
4. combine the results from step 2) and 3) to obtain a crystal structure.

Usually, we also have to check the `volume` and `atomic distances` of generated structure, only keep the structures which have reasonable `volume` and `atomic distances`.

To facilitate all these jobs, our `crystallus` library provides three modules:

* `WyckoffCfgGenerator`: generate possible Wyckoff configurations for the given space group and composition of primitive cell.
* `CrystalGenerator`: generate crystal structures for the given space group and Wyckoff configurations.
* `WyckoffDB, SpaceGroupDB`: database include space group and corresponding Wyckoff information.

The folloing content shows how to use `crystallus`.

### 1. generate Wyckoff configurations

As an example, we will try to generate structures for `Ca2C2O6`. The true space group of this structure is `167`, and the Wyckoff configuration is `{Ca: 2b, C: 2a, O: 6e}`.
You can use `SpaceGroupDB` to get the information of Wyckoff position about space group `167`.

In [2]:
from crystallus import SpaceGroupDB

wys = SpaceGroupDB.get(spacegroup_num=167).wyckoffs
[{'Wyckoff letter': w.letter, 'multiplicity': w.multiplicity, 'reusable': w.reuse, 'Wyckoff position': w.positions} for w in wys ]

[{'Wyckoff letter': 'f',
  'multiplicity': 12,
  'reusable': True,
  'Wyckoff position': '(x,y,z), (z,x,y), (y,z,x), (-y+1/2,-x+1/2,-z+1/2), (-x+1/2,-z+1/2,-y+1/2), (-z+1/2,-y+1/2,-x+1/2), (-x,-y,-z), (-z,-x,-y), (-y,-z,-x), (y+1/2,x+1/2,z+1/2), (x+1/2,z+1/2,y+1/2), (z+1/2,y+1/2,x+1/2)'},
 {'Wyckoff letter': 'e',
  'multiplicity': 6,
  'reusable': True,
  'Wyckoff position': '(x,-x+1/2,1/4), (1/4,x,-x+1/2), (-x+1/2,1/4,x), (-x,x+1/2,3/4), (3/4,-x,x+1/2), (x+1/2,3/4,-x)'},
 {'Wyckoff letter': 'd',
  'multiplicity': 6,
  'reusable': False,
  'Wyckoff position': '(1/2,0,0), (0,1/2,0), (0,0,1/2), (1/2,0,1/2), (0,1/2,1/2), (1/2,1/2,0)'},
 {'Wyckoff letter': 'c',
  'multiplicity': 4,
  'reusable': True,
  'Wyckoff position': '(x,x,x), (-x+1/2,-x+1/2,-x+1/2), (-x,-x,-x), (x+1/2,x+1/2,x+1/2)'},
 {'Wyckoff letter': 'b',
  'multiplicity': 2,
  'reusable': False,
  'Wyckoff position': '(0,0,0), (1/2,1/2,1/2)'},
 {'Wyckoff letter': 'a',
  'multiplicity': 2,
  'reusable': False,
  'Wyckoff position

Let's generate some possible Wyckoff configurations for the composition `Ca2C2O6` under space group `167`.

In [3]:
from crystallus import WyckoffCfgGenerator

WyckoffCfgGenerator?

Init signature:
WyckoffCfgGenerator(
    *,
    max_recurrent: int = 1000,
    n_jobs: int = -1,
    priority: Optional[Dict[int, Dict[str, float]]] = None,
    **composition,
)
Docstring:      <no docstring>
Init docstring:
A generator for possible Wyckoff configuration generation.

Parameters
----------
max_recurrent : int, optional
    Max recurrent until generate a reasonable structure, by default 5_000
n_jobs : int, optional
    Number of cpu cores when parallel calculation, by default -1
priority:
    Priorities for Wyckoff position (WP). WP selection is equal to sampling
    from an Uniform distribution of the pool of all availables.
    By default, all possible WPs have a priority value `1`.
    Give this parameter will overwrite the corresponding WP priority value.
    For example, space group 167 has WP `[a, b, c, d, e, f]`
    By default, all their priority values are `1`. Now, we want to lift the
    priority of PW `a`, `b` and `d` and never use `e`, we can give parameter `

In [4]:
from crystallus.crystallus import WyckoffCfgGenerator

In [5]:
WyckoffCfgGenerator?

Init signature: WyckoffCfgGenerator(max_recurrent, n_jobs, **composition)
File:           ~/mambaforge/envs/crystallus/lib/python3.9/site-packages/crystallus/__init__.py
Type:           type
Subclasses:     


In [7]:
composition = {'Ca': 2, 'C': 2, 'O': 6}
wyg = WyckoffCfgGenerator(**composition)

In [9]:
wyg.max_recurrent

1000

In [22]:
wyg

In [5]:
composition = {'Ca': 2, 'C': 2, 'O': 6}

wyg = WyckoffCfgGenerator(**composition)
wyg

AttributeError: 'crystallus.WyckoffCfgGenerator' object has no attribute 'n_jobs'

You have noticed that the minimum input for the initialization of a `WyckoffCfgGenerator` is just a composition.
Now, we can try to use this generator to generate Wyckoff configuration(s). First, let's try to generate one. This can be done by the `gen_one` method.

In [15]:
wyg.gen_one?

Signature: wyg.gen_one(spacegroup_num)
Docstring: <no docstring>
Type:      builtin_function_or_method


In [16]:
cfg = wyg.gen_one(spacegroup_num=167)
cfg

{'C': ['b'], 'Ca': ['a'], 'O': ['d']}

If everything goes well, the above cell will return a dict contains something like: `{'C': ['b'], 'Ca': ['a'], 'O': ['d']}`.
Here, `C`, `Ca`, and `O` are the element names. All elements are sorted by the alphabet. The `['b']`, `['a']`, and `['d']` are the corresponding Wyckoff positions which are provided by Wyckoff letters.

Maybe you are confused that the return of this method is not unique. That makes sense because under space group `167`, there are four possible configurations for the composition `Ca2C2O6`. Call the `gen_one` method will execute a random search in all possible configurations. When it finds one, it returns the result and stops searching. This means if you want to get more configurations, you should call the `gen_one` method many times. We know that for almost all the cases, the possible configurations are not one, to simplify your works, we provide the `gen_many` method.

In [17]:
wyg.gen_many?

Signature: wyg.gen_many(size, /, *spacegroup_num)
Docstring: <no docstring>
Type:      builtin_function_or_method


In [18]:
cfgs = wyg.gen_many(100, 167)
cfgs

[{'C': ['a'], 'Ca': ['b'], 'O': ['d']},
 {'C': ['b'], 'Ca': ['a'], 'O': ['d']},
 {'C': ['b'], 'Ca': ['a'], 'O': ['e']},
 {'C': ['a'], 'Ca': ['b'], 'O': ['e']}]

You can calculate more multiply space group in one call. Just list space group numbers as `*` parameters. In this case, the return will be a dict with space group number as key and configuration list as value. For example, if our space group candidate are `[194, 148, 167]`, you can call `gen_many` like this:

In [9]:
%%time

cfgs = wyg.gen_many(20, 194, 148, 167)
cfgs

CPU times: user 9.2 ms, sys: 3.44 ms, total: 12.6 ms
Wall time: 2.61 ms


{194: [{'C': ['a'], 'Ca': ['d'], 'O': ['g']},
  {'C': ['d'], 'Ca': ['c'], 'O': ['g']},
  {'C': ['d'], 'Ca': ['c'], 'O': ['b', 'e']},
  {'C': ['b'], 'Ca': ['d'], 'O': ['g']},
  {'C': ['b'], 'Ca': ['c'], 'O': ['h']},
  {'C': ['c'], 'Ca': ['d'], 'O': ['g']},
  {'C': ['c'], 'Ca': ['a'], 'O': ['b', 'e']},
  {'C': ['c'], 'Ca': ['b'], 'O': ['a', 'f']},
  {'C': ['d'], 'Ca': ['a'], 'O': ['c', 'e']},
  {'C': ['b'], 'Ca': ['d'], 'O': ['c', 'f']},
  {'C': ['b'], 'Ca': ['c'], 'O': ['g']},
  {'C': ['a'], 'Ca': ['c'], 'O': ['h']},
  {'C': ['a'], 'Ca': ['c'], 'O': ['d', 'f']},
  {'C': ['b'], 'Ca': ['d'], 'O': ['h']},
  {'C': ['a'], 'Ca': ['c'], 'O': ['d', 'e']},
  {'C': ['b'], 'Ca': ['d'], 'O': ['a', 'e']},
  {'C': ['d'], 'Ca': ['a'], 'O': ['h']}],
 148: [{'C': ['c'], 'Ca': ['a', 'b'], 'O': ['d', 'e']},
  {'C': ['a', 'b'], 'Ca': ['c'], 'O': ['f']},
  {'C': ['c'], 'Ca': ['c'], 'O': ['f']},
  {'C': ['a', 'b'], 'Ca': ['c'], 'O': ['c', 'c', 'c']},
  {'C': ['c'], 'Ca': ['c'], 'O': ['a', 'b', 'c', 'c']}],
 

`gen_many_iter` is an iterative version of `gen_many`. You can use this method to render a progress bar during generation, or something else you want.

In [10]:
%%time

from tqdm.notebook import tqdm

space_group_cans = [194, 148, 167, 161, 11, 12, 65, 140, 225]

with tqdm(total=len(space_group_cans)) as pbar:
    for spacegroup_num, cfg_list in wyg.gen_many_iter(5000, *space_group_cans):
        print(f'space group: {spacegroup_num}, size of generated samples: {len(cfg_list)}')
        pbar.update()

space group: 194, size of generated samples: 72
space group: 148, size of generated samples: 14
space group: 167, size of generated samples: 4
space group: 161, size of generated samples: 2
space group: 11, size of generated samples: 199
space group: 12, size of generated samples: 2441
space group: 65, size of generated samples: 4307
space group: 140, size of generated samples: 96
space group: 225, size of generated samples: 4

CPU times: user 34 s, sys: 69.4 ms, total: 34.1 s
Wall time: 4.61 s


### 2. generate crystal structures

We have generated some Wyckoff configurations, the next is consuming the Wyckoff configurations to generate crystal structures. To facilitate the task, we provide the `CrystalGenerator` class.

In [19]:
from crystallus import CrystalGenerator

CrystalGenerator?

Init signature:
CrystalGenerator(
    spacegroup_num: int,
    volume_of_cell: float,
    variance_of_volume: float,
    *,
    angle_range: Tuple[float, float] = (30.0, 150.0),
    angle_tolerance: float = 20.0,
    lattice: Optional[Tuple[float]] = None,
    empirical_coords: Optional[List[Tuple[str, List[float]]]] = None,
    empirical_coords_variance: float = 0.01,
    empirical_coords_sampling_rate: float = 1.0,
    empirical_coords_loose_sampling: bool = True,
    max_attempts_number: int = 5000,
    n_jobs: int = -1,
    verbose: bool = False,
)
Docstring:      <no docstring>
Init docstring:
A generator for possible crystal structure generation.

Parameters
----------
spacegroup_num:
    Specify the spacegroup.
volume_of_cell:
    The estimated volume of primitive cell. Unit is Å^3.
variance_of_volume:
    The estimated variance of volume prediction. Unit is Å^3.
    ``volume_of_cell`` and ``variance_of_volume`` will be used to build
    a Gaussion distribution for the sampling 

To initialize a `CrystalGenerator`, parameter `spacegroup_num`, `volume_of_cell` of primitive cell and the `variance_of_volume` are needed at least.

In [20]:
volume_of_cell = 127.170256
variance_of_volume = 20.
sp_num = 167

cg = CrystalGenerator(sp_num, volume_of_cell, variance_of_volume)
cg

AttributeError: 'CrystalGenerator' object has no attribute 'verbose'

Like the `WyckoffCfgGenerator`, there are also `gen_one`, `gen_many`, and `gen_may_iter` methods attached with `CrystalGenerator` object. Let's ues the `gen_one` method for a quick try.

In [13]:
cg.gen_one?

Signature:
cg.gen_one(
    *,
    check_distance: bool = True,
    distance_scale_factor: float = 0.1,
    **cfg: Dict[str, Tuple[str]],
)
Docstring:
Try to generate a legal crystal structure with given configuration set.

Parameters
----------
check_distance: bool, optional
    Whether the atomic distance should be checked. default ``True``
distance_scale_factor : float, optional
    Scale factor to determine the tolerance of atomic distances when distance checking. Unit is Å,
    When ``check_distance`` is ``True``, Any structure has
    all_atomic_distance < (A_atom_covalent_radius + B_atom_covalent_radius) * (1 - distance_scale_factor) will be rejected,
    by default 0.1
**cfg: Dict[str, Tuple[str]]
    Wyckoff Configuration set, which is a dict with format like:
    {"Li": ["a", "c"], "O": ["i"]}. Here, the "Li" is an available element
    symbol and ["a", "c"] is a list which contains coresponding Wyckoff
    letters. For convenience, dict will be sorted by keys.

Returns
------

All `gen_xxx` methods consume Wyckoff configurations to generate crystal structures. Please note the parameter `distance_scale_factor`, generator use this parameter to determine the acceptabel atomic distance. Here is the accept condition:
> distance between atom `a` and `b` > (radius of `a` + radius of `b`) x (1 – `distance_scale_factor`).

If the generator cannot generate any structure after multiple attempts, you can try to relax the atomic distance constraint by increasing this parameter.

In [22]:
%%time

cfgs[sp_num][0]

raw_s = cg.gen_one(**cfgs[sp_num][0])
raw_s

CPU times: user 101 µs, sys: 1e+03 ns, total: 102 µs
Wall time: 105 µs


{'spacegroup_num': 167,
 'volume': 120.08597186492635,
 'lattice': [[4.932912846778897, 0.0, 0.3835001248470154],
  [0.35484281230671005, 4.920133710832515, 0.3835001248470154],
  [0.0, 0.0, 4.947797641342455]],
 'species': ['C', 'C', 'Ca', 'Ca', 'O', 'O', 'O', 'O', 'O', 'O'],
 'wyckoff_letters': ['b', 'b', 'a', 'a', 'e', 'e', 'e', 'e', 'e', 'e'],
 'coords': [[0.0, 0.0, 0.0],
  [0.5, 0.5, 0.5],
  [0.25, 0.25, 0.25],
  [0.75, 0.75, 0.75],
  [0.8205210681590107, 0.6794789318409893, 0.25],
  [0.25, 0.8205210681590107, 0.6794789318409893],
  [0.6794789318409893, 0.25, 0.8205210681590107],
  [0.1794789318409893, 0.3205210681590107, 0.75],
  [0.75, 0.1794789318409893, 0.3205210681590107],
  [0.3205210681590107, 0.75, 0.1794789318409893]]}

The result is a dict contains `species`, `lattice`, `coords` and other information. These information can be used to build the  `pymatgen.Structure` and `ase.Structure` object.

In [15]:
from pymatgen import Structure

s = Structure(lattice=raw_s['lattice'], species=raw_s['species'], coords=raw_s['coords'])
s

Structure Summary
Lattice
    abc : 5.4596917915621965 5.4596917915621965 5.4596917915621965
 angles : 85.05026616421641 85.05026616421641 85.05026616421641
 volume : 161.02187020709744
      A : 5.439331385109984 0.0 0.47107169497766643
      B : 0.4320379763826813 5.422146171397969 0.47107169497766643
      C : 0.0 0.0 5.4596917915621965
PeriodicSite: C (1.4678, 1.3555, 1.6005) [0.2500, 0.2500, 0.2500]
PeriodicSite: C (4.4035, 4.0666, 4.8014) [0.7500, 0.7500, 0.7500]
PeriodicSite: Ca (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Ca (2.9357, 2.7111, 3.2009) [0.5000, 0.5000, 0.5000]
PeriodicSite: O (3.4023, 5.1508, 2.0715) [0.5500, 0.9500, 0.2500]
PeriodicSite: O (1.5975, 2.9824, 5.5634) [0.2500, 0.5500, 0.9500]
PeriodicSite: O (5.2752, 1.3555, 3.5683) [0.9500, 0.2500, 0.5500]
PeriodicSite: O (2.4691, 0.2713, 4.3303) [0.4500, 0.0500, 0.7500]
PeriodicSite: O (4.2739, 2.4398, 0.8385) [0.7500, 0.4500, 0.0500]
PeriodicSite: O (0.5962, 4.0666, 2.8335) [0.0500, 0.7500, 0.45

The following is batched generation.

In [16]:
%%time

raw_ss = cg.gen_many(100, *cfgs[sp_num])

print(f"type of raw_ss: {raw_ss.__class__}, size: {len(raw_ss)}")

type of raw_ss: <class 'tuple'>, size: 131
CPU times: user 17.3 ms, sys: 3.69 ms, total: 21 ms
Wall time: 5.32 ms


Also, the iterative version

In [17]:
%%time

with tqdm(total=len(cfgs[sp_num])) as pbar:
    for cfg, structures in cg.gen_many_iter(500, *cfgs[sp_num]):
        print(f'configuration: {cfg}, size of structures: {len(structures)}')
        pbar.update()

configuration: {'C': ['a'], 'Ca': ['b'], 'O': ['e']}, size of structures: 127
configuration: {'C': ['b'], 'Ca': ['a'], 'O': ['e']}, size of structures: 82
configuration: {'C': ['b'], 'Ca': ['a'], 'O': ['d']}, size of structures: 18
configuration: {'C': ['a'], 'Ca': ['b'], 'O': ['d']}, size of structures: 449

CPU times: user 123 ms, sys: 10.5 ms, total: 133 ms
Wall time: 66.9 ms
